# Capstone project - Q4

 Can a predictive model be built for future prediction of the possibility of complaints of the specific type that you identified in response to Question 1?

On Q3 I trained a Random Tree Forest classifier for identifying which are the most related features to certain complaint type-

    The Random Tree Forest can also be used for predicting. We can use the test data for seeing how good it is as a classifier/predictor

In [1]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

feature_list = ['BldgArea', 'BldgDepth', 'BuiltFAR', 'CommFAR', 'FacilFAR', 'Lot', 'LotArea', 'LotDepth', 'NumBldgs', 'NumFloors', 'OfficeArea', 'ResArea', 'ResidFAR', 'RetailArea', 'YearBuilt', 'YearAlter1']


def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_dd7eba6d38744f2ba0e41e1f24c776e4 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='***',
    ibm_auth_endpoint="https://iam.eu-gb.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.eu-geo.objectstorage.service.networklayer.com')

body0 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='BK_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body0, "__iter__"): body0.__iter__ = types.MethodType( __iter__, body0 )
df_plutonBK = pd.read_csv(body0, usecols=['Address']+feature_list+['ZipCode', 'YCoord', 'XCoord'])

body1 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='MN_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body1, "__iter__"): body1.__iter__ = types.MethodType( __iter__, body1 )
df_plutonMN = pd.read_csv(body1, usecols=['Address']+feature_list+['ZipCode', 'YCoord', 'XCoord'])

body2 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='QN_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body2, "__iter__"): body2.__iter__ = types.MethodType( __iter__, body2 )
df_plutonQN = pd.read_csv(body2, usecols=['Address']+feature_list+['ZipCode', 'YCoord', 'XCoord'])

body3 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='SI_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body3, "__iter__"): body3.__iter__ = types.MethodType( __iter__, body3 )
df_plutonSI = pd.read_csv(body3, usecols=['Address']+feature_list+['ZipCode', 'YCoord', 'XCoord'])

body4 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='BX_18v1.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body4, "__iter__"): body4.__iter__ = types.MethodType( __iter__, body4 )
df_plutonBX = pd.read_csv(body4, usecols=['Address']+feature_list+['ZipCode', 'YCoord', 'XCoord'])

df_pluton = df_plutonBK.append(df_plutonBX).append(df_plutonMN).append(df_plutonQN).append(df_plutonSI)

body5 = client_dd7eba6d38744f2ba0e41e1f24c776e4.get_object(Bucket='capstoneproject-donotdelete-pr-w1e8lgzafyovbk',Key='fhrw-4uyv.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body5, "__iter__"): body5.__iter__ = types.MethodType( __iter__, body5 )

df_131 = pd.read_csv(body5)

In [2]:
## THIS PART COMES FROM Q3

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Selecting the columns we are interested in
df = df_131[['incident_address','complaint_type']].copy() 
#Cleaning the dataFrame
indexDrop = df.loc[(df['complaint_type'] == 'HPD Literature Request')].index 
df.drop(indexDrop,inplace=True)
# Dropping rows with empty cell
df.dropna(inplace=True)
# Dropping duplicated complaints for 
df.drop_duplicates(subset=['incident_address','complaint_type'],inplace=True)
#Setting the index to incient_adrres so we can join the df_131 and df_pluton
df.set_index('incident_address',inplace=True)
df_131Pluton = df.join(other=df_pluton.set_index('Address'),how='inner')
df_tree = df_131Pluton.loc[(df_131Pluton['complaint_type'] == 'GENERAL CONSTRUCTION') | ( df_131Pluton['complaint_type']=='PAINT - PLASTER') | ( df_131Pluton['complaint_type']=='PLUMBING')]

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
features = df_tree[feature_list]
labels =  df_tree['complaint_type']
codes_labels, uniques_labels = pd.factorize(labels)
train_features, test_features, train_labels, test_labels = train_test_split(features,codes_labels, test_size = 0.25, random_state = 42)
# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(train_features, train_labels);

In [3]:
# Use the forest's predict method on the test data
predictions = rf.predict(test_features)
# Calculate the absolute errors
errors = abs(predictions - test_labels)
# Print out the mean absolute error (mae)
print('Mean Absolute Error:', round(np.mean(errors), 2), 'degrees.')

Mean Absolute Error: 0.91 degrees.
